In [1]:
!pip install -U tensorflow-gpu

     |████████████████████████████████| 380.8MB 49kB/s 
     |████████████████████████████████| 3.8MB 37.3MB/s 
     |████████████████████████████████| 450kB 42.7MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=1e9059a9ef057a33f8ab5b12dad377d32bd5ce4788bb728ff60f569b6d69a96c
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 1.14.0 has requirement tensorboard<1.15.0,>=1.14.0, but you'll have tensorboard 2.0.0 which is incompatible.
ERROR: tensorflow 1.14.0 has requirement tensorflow-estimator<1.15.0rc0,>=1.14.0rc0, but you'll have tensorflow-estimator 2.0.0 which is incompatible.
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tens

In [2]:
import tensorflow as tf
from PIL import Image
tf.__version__

'2.0.0'

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_train = (x_train-127.5)/127.5
x_test = (x_test-127.5)/127.5

170500096/170498071 [==============================] - 4s 0us/step


In [4]:
x_train.shape

(50000, 32, 32, 3)

In [5]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds

<TensorSliceDataset shapes: ((32, 32, 3), (1,)), types: (tf.float64, tf.uint8)>

In [6]:
train_ds=train_ds.shuffle(10000)
train_ds

<ShuffleDataset shapes: ((32, 32, 3), (1,)), types: (tf.float64, tf.uint8)>

In [7]:
train_ds=train_ds.batch(32)
train_ds

<BatchDataset shapes: ((None, 32, 32, 3), (None, 1)), types: (tf.float64, tf.uint8)>

In [0]:
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.shuffle(10000).batch(32)

In [0]:
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten, Softmax
from tensorflow.keras.models import Model

class Mymodel(Model):
    def __init__(self):
        super().__init__()
        self.d1=Conv2D(32,(3,3),activation='relu',input_shape=(32,32,3))
        self.d2=MaxPool2D((2,2))
        self.d3=Dense(256,activation='relu')
        self.d4=Conv2D(64,(3,3),activation='relu')
        self.d5=Conv2D(128,(3,3),activation='relu')
        self.d6=Conv2D(128,(3,3),activation='relu')
        self.flatten=Flatten()
        self.dropout=Dropout(0.3)
        self.dropoutmore=Dropout(0.7)
        self.softmax=Dense(10,activation='softmax')
    def call(self, x): 
        x = self.d1(x)
        x = self.d2(x)
        x = self.dropout(x)
        x = self.d4(x)
        x = self.d2(x)
        #x = self.d4(x)
        #x = self.d2(x)
        x = self.dropout(x)
        x = self.d5(x)
        #x = self.d2(x)
        x = self.d6(x)
#        x = self.d6(x)
        x = self.d2(x)
        x = self.dropout(x)
        x = self.flatten(x)
        x = self.d3(x)
        x = self.dropoutmore(x)
        x = self.softmax(x)
        return x

model=Mymodel()


In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [0]:
#@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [0]:
#@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [0]:
EPOCHS = 30

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

  # Reset the metrics for the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 1.4391406774520874, Accuracy: 47.1619987487793, Test Loss: 1.1336320638656616, Test Accuracy: 59.82999801635742
Epoch 2, Loss: 1.0214722156524658, Accuracy: 63.61000061035156, Test Loss: 0.9524996280670166, Test Accuracy: 66.45999908447266
Epoch 3, Loss: 0.8426815271377563, Accuracy: 70.24600219726562, Test Loss: 0.8818418979644775, Test Accuracy: 69.30000305175781
Epoch 4, Loss: 0.7322424650192261, Accuracy: 74.28600311279297, Test Loss: 0.8513899445533752, Test Accuracy: 70.910003662109

In [0]:
model.summary()

Model: "mymodel_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           multiple                  896       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 multiple                  0         
_________________________________________________________________
dense_18 (Dense)             multiple                  256       
_________________________________________________________________
conv2d_30 (Conv2D)           multiple                  18496     
_________________________________________________________________
conv2d_31 (Conv2D)           multiple                  73856     
_________________________________________________________________
conv2d_32 (Conv2D)           multiple                  147584    
_________________________________________________________________
flatten_9 (Flatten)          multiple                  0 